# Produksjonssystem for FNR

Det første vi gjør er å laste inn nødvendige biblioteker
* matplotlib for plotting
* ipywidgets og IPython for menyer
* toml for å laste inn filer med opsjoner

I tillegg laster vi inn FNR-klassen fra fnr.py

In [ ]:
import pandas as pd
import toml
from fnr_plott_og_tabell import *

import sys
sys.path.append('py')
import fnr as fnr

%matplotlib inline

# Oppsett av FNR-klassen
Vi setter først opp sti til data og kataloger (for aggregering).
I tillegg setter vi opp en liste med alle fylker vi ønsker å ha med samt aggregeringer som ikke ligger i omkodingskatalogen.
Vi laster dette inn fra en toml-fil i samme mappe.

In [ ]:
config = toml.load('config.toml')
mappings = config.get('mappings')
fylker = config.get('fylker').get('fylker')
data = config.get('filstier').get('data')
katalog = config.get('filstier').get('katalog')

Deretter initialiserer vi FNR-klassen og laster inn data for ønskede årganger. Under vises også hvordan man laster inn en ekstra årgang.

In [ ]:
fnr = fnr.fnr_class(2008, 2019, {'lists': ['pubagg', 'a38np'], 'mappings': mappings}, fylker, data, katalog)
fnr.add_year(2020)

Nå har vi en "instance"/et tilfelle av FNR-klassen. En egenskap ved denne er DataFrame'en med alle dataene.

In [ ]:
fnr.return_selection('pubagg', [2020], ['bnp'], ['2x35', '2x41_43'], [], wide_by='fylke', round_to=0)

# Plotting
FNR-klassen har to metoder (funksjoner) man kan benytte seg av. Den ene heter **return_selecion()**. Man kan skrive *help(return_selection)* for se hvordan denne brukes.
Metoden **return_selection()** kan brukes til å hente ut data og foreksemepel plotte disse som vist under.

In [ ]:
plott_fylke(fnr.df, fnr)

In [ ]:
plott_naering(fnr.df, fnr.agg_dict, fnr)

In [ ]:
plott_naering2(fnr.df, fnr.agg_dict, fnr)

# Tabulering av data
Man kan også bruke metoden **return_selection()** til å tabulere valgte data.

In [ ]:
lag_tabell(fnr.df, fnr.agg_dict, fnr)

# Prikking av data
Den andre metoden i klassen heter **supress_data()**. Denne prikker data (setter verdiene til "not a number/nan"). Den tar i mot en dictionary med nøklene årgang og en liste med par av næringer og fylker som skal prikkes.

In [ ]:
fnr.suppress_data({2020: [['2x90_97', 'f54'], ['2x85', 'f30']]})
fnr.df

# Ekstra
Eksempel på bruk av **return_selection()**.

In [ ]:
fnr.return_selection('pubagg', [], ['bnp'], [], None, round_to=0, wide_by='fylke', first_diff=True)

In [ ]:
fnr.return_selection('pubagg', [2019,2020], ['bnp'], ['2x35', '2x41_43'], ['f30', 'f03', 'f34'], wide_by='aggregat', round_to=0, sort_by_row=[pd.Period('2020'), 'bnp', 'f03'])

In [ ]:
fnr.return_selection('pubagg', [2019,2020], ['bnp'], ['2x35', '2x41_43'], ['f30', 'f03', 'f34'], wide_by='årgang', round_to=0, sort_by=['2020'])